In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import ndcg_score, average_precision_score

import plotly.express as px

# Avaliando ranking baseado em arquivos

Para rodar este notebook via linhe de comando utilizar:

> jupyter nbconvert --execute --to html Sprint_1_Job_métricas_de_ranking.ipynb

Trocar também o path dos arquivos **ground_truth.csv** e o **search_ranking.csv**.



## Carregando dados

In [ ]:
ground_truth_filepath = "../../dados/regis/regis_ground_truth.csv"
search_ranking_filepath = "../../dados/regis/search_ranking.csv"

In [ ]:
ground_truth = pd.read_csv(ground_truth_filepath)
ground_truth.head()

In [ ]:
rankings = pd.read_csv(search_ranking_filepath)
rankings.head()

In [ ]:
validation_dataset = rankings.merge(
    ground_truth,
    on=["query_id", "document_id"],
    suffixes=("_ranking", "_ground_truth"),
    how="outer"
).assign(
    evaluated = lambda row: row.relevance_ground_truth.notnull(),
    irrelevant_ground_truth = lambda row: (row.relevance_ranking.isnull()) & (row.relevance_ground_truth == 0),
).query(
    "irrelevant_ground_truth == False"
).fillna(
    value={
        "relevance_ranking": 0,
        "relevance_ground_truth": 0
    }
).drop(
    columns="irrelevant_ground_truth"
)

validation_dataset.head()

# Avaliando métricas

## Criando métricas

In [ ]:
metrics = {
    "query_id": validation_dataset.query_id.unique(),
    "ndcg": list(),
    "ap": list(),
    "eval_prop": list(),
}

thresh = 1
ndcgs = list()
for qid in metrics["query_id"]:
    query_dataset = validation_dataset.query("query_id == @qid")
    # ndcg
    metrics["ndcg"].append(
        ndcg_score([query_dataset["relevance_ground_truth"]], [query_dataset["relevance_ranking"]])
    )
    # average precision
    metrics["ap"].append(
        average_precision_score(
            [0 if r <= thresh else 1 for r in query_dataset["relevance_ground_truth"]],
            query_dataset["relevance_ranking"]
        )
    )

    # evaluated proportion
    metrics["eval_prop"].append(
        query_dataset.groupby(
            "query_id"
        ).aggregate({
            "evaluated": ["sum", "count"],
        }).reset_index(
        )["evaluated"].assign(
            evaluated_proportion = lambda row: (row["sum"] / row["count"])
        )["evaluated_proportion"][0]
    )


metrics_df = pd.DataFrame(metrics)

metrics_df.head()

## Avaliando métricas

### NDCG

In [ ]:
avg_ndcg = np.mean(metrics_df.ndcg)
print("NDCG médio: {}".format(avg_ndcg))

In [ ]:
fig = px.histogram(
    metrics_df,
    x="ndcg",
).add_vline(
    x=avg_ndcg,
    line_color="red",
).update_layout(
    title="Distribuição dos NDGCs das queries com indicação da média",
    xaxis_title="NDCG",
    yaxis_title="Contagem",
)
fig.show()

### Average Precisions

In [ ]:
map = np.mean(metrics_df.ap)
print("MAP: {}".format(map))

In [ ]:
fig = px.histogram(
    metrics_df,
    x="ap",
).add_vline(
    x=map,
    line_color="red"
).update_layout(
    title="Distribuição dos Average Precisions das queries com indicação do Mean Average Precision",
    xaxis_title="Average Precision",
    yaxis_title="Contagem",
)
fig.show()

### Porcentagem de documentos avaliados

In [ ]:
average_eval_docs = np.mean(metrics_df.eval_prop)
print("Média de documentos avaliados por query: {}".format(average_eval_docs))

In [ ]:
fig = px.histogram(
    metrics_df,
    x="eval_prop",
).add_vline(
    x=average_eval_docs,
    line_color="red"
).update_layout(
    title="Distribuição dos percentuais de documentos avaliados com indicação da média",
    xaxis_title="Percentual de documentos avaliados",
    xaxis_tickformat=".0%",
    yaxis_title="Contagem",
)
fig.show()

### Métricas por Query

In [ ]:
data_viz = metrics_df.melt(
    id_vars=["query_id"],
    value_vars=["ndcg", "ap", "eval_prop"],
    var_name="metric"
)

fig = px.bar(
    data_viz, x="query_id", y="value", color="metric",
    barmode="group", color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.show()

In [ ]:
fig = px.parallel_coordinates(
    metrics_df, color="eval_prop",
    color_continuous_scale=px.colors.sequential.YlOrRd,
    width=1000
)

fig.show()
